In [50]:
import json
import os
from dotenv import load_dotenv
from termcolor import colored  

from openai import OpenAI

load_dotenv()

GPT_MODEL = "gpt-3.5-turbo-0613"
client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

In [34]:
def chat_completion_request(messages, tools=None, tool_choice="auto", model=GPT_MODEL):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice=tool_choice,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e


In [35]:
def pretty_print_conversation(messages):
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "function": "magenta",
    }
    
    for message in messages:
        if message["role"] == "system":
            print(colored(f"system: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "user":
            print(colored(f"user: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and message.get("function_call"):
            print(colored(f"assistant: {message['function_call']}\n", role_to_color[message["role"]]))
        elif message["role"] == "assistant" and not message.get("function_call"):
            print(colored(f"assistant: {message['content']}\n", role_to_color[message["role"]]))
        elif message["role"] == "function":
            print(colored(f"function ({message['name']}): {message['content']}\n", role_to_color[message["role"]]))


In [43]:
tools = [{
        "type": "function",
        "function": {
            "name": "extract_people_information",
            "description": "Extract the information for a person. Make sure to use MM/DD/YY formatting for dates.",
            "parameters": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string",
                        "description": "Name of the person to be extracted."
                    },
                    "birthday": {
                        "type": "string",
                        "description": "Date of birth of the person to be extracted."
                    },
                    "location": {
                        "type": "string",
                        "description": "Location of the person to be extracted."
                    }
                },
                "required": ["name, birthday, location"],
            },
        }
    }
]

In [51]:
messages = []
messages.append({"role": "system", "content": "You're an assistant that summarizes the information for all people mentioned in the user messages. You should make sure you don't miss anyone that is mentioned. There will be multiple people in each message."})
messages.append({"role": "user", "content": "I met Alessio, a guy from Rome whose birthday is on Aug 29th and is 29 years old (we are in 2024 now). I also met Sarah who was born on the 8th of August instead and is from Milwaukee"})
chat_response = chat_completion_request(messages, tools)
print(chat_response.choices)
assistant_message = chat_response.choices[0].message
assistant_message.content = str(assistant_message.tool_calls[0].function)
messages.append({"role": assistant_message.role, "content": assistant_message.content})

[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_epAZsrsTCfiiyD9c8JqngHJY', function=Function(arguments='{\n  "name": "Alessio",\n  "birthday": "08/29/1995",\n  "location": "Rome"\n}', name='extract_people_information'), type='function')]))]


In [52]:
pretty_print_conversation(messages)

print(assistant_message.tool_calls[0].function.arguments)

system: You're an assistant that summarizes the information for all people mentioned in the user messages. You should make sure you don't miss anyone that is mentioned. There will be multiple people in each message.

user: I met Alessio, a guy from Rome whose birthday is on Aug 29th and is 29 years old (we are in 2024 now). I also met Sarah who was born on the 8th of August instead and is from Milwaukee

assistant: Function(arguments='{\n  "name": "Alessio",\n  "birthday": "08/29/1995",\n  "location": "Rome"\n}', name='extract_people_information')

{
  "name": "Alessio",
  "birthday": "08/29/1995",
  "location": "Rome"
}


In [56]:
tools = [{
    "type": "function",
    "function": {
        "name": "extract_people_information",
        "description": "Extract information for multiple people. Make sure to use MM/DD/YY formatting for dates.",
        "parameters": {
            "type": "object",
            "properties": {
                "people": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "properties": {
                            "name": {
                                "type": "string",
                                "description": "Name of the person to be extracted."
                            },
                            "birthday": {
                                "type": "string",
                                "description": "Date of birth of the person to be extracted."
                            },
                            "location": {
                                "type": "string",
                                "description": "Location of the person to be extracted."
                            }
                        },
                        "required": ["name", "birthday", "location"]
                    }
                }
            },
            "required": ["people"]
        },
    }
}]


In [57]:
messages = []
messages.append({"role": "system", "content": "You're an assistant that summarizes the information for all people mentioned in the user messages. You should make sure you don't miss anyone that is mentioned. There will be multiple people in each message."})
messages.append({"role": "user", "content": "I met Alessio, a guy from Rome whose birthday is on Aug 29th and is 29 years old (we are in 2024 now). I also met Sarah who was born on the 8th of August instead and is from Milwaukee"})
chat_response = chat_completion_request(messages, tools)
print(chat_response.choices)
assistant_message = chat_response.choices[0].message
assistant_message.content = str(assistant_message.tool_calls[0].function)
messages.append({"role": assistant_message.role, "content": assistant_message.content})

[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_QZz6WpENZhxTLCw2sok8p2qh', function=Function(arguments='{\n  "people": [\n    {\n      "name": "Alessio",\n      "birthday": "08/29/95",\n      "location": "Rome"\n    },\n    {\n      "name": "Sarah",\n      "birthday": "08/08/95",\n      "location": "Milwaukee"\n    }\n  ]\n}', name='extract_people_information'), type='function')]))]


In [58]:
pretty_print_conversation(messages)

print(assistant_message.tool_calls[0].function.arguments)

system: You're an assistant that summarizes the information for all people mentioned in the user messages. You should make sure you don't miss anyone that is mentioned. There will be multiple people in each message.

user: I met Alessio, a guy from Rome whose birthday is on Aug 29th and is 29 years old (we are in 2024 now). I also met Sarah who was born on the 8th of August instead and is from Milwaukee

assistant: Function(arguments='{\n  "people": [\n    {\n      "name": "Alessio",\n      "birthday": "08/29/95",\n      "location": "Rome"\n    },\n    {\n      "name": "Sarah",\n      "birthday": "08/08/95",\n      "location": "Milwaukee"\n    }\n  ]\n}', name='extract_people_information')

{
  "people": [
    {
      "name": "Alessio",
      "birthday": "08/29/95",
      "location": "Rome"
    },
    {
      "name": "Sarah",
      "birthday": "08/08/95",
      "location": "Milwaukee"
    }
  ]
}
